In [128]:
import pandas as pd
import datetime as dt

## Data Understanding
- Load dataset
---





## Data Understanding
- Preview dataset
---





In [129]:
df_customer = pd.read_csv('../assets/jobsheet-5/customers.csv')
df_customer.head()

,CustomerID,Segment,Currency
0,3,SME,EUR
1,5,LAM,EUR
2,6,SME,EUR
3,7,LAM,EUR
4,9,SME,EUR


In [130]:
df_gasstation = pd.read_csv('../assets/jobsheet-5/gasstations.csv')
df_gasstation.head()

,GasStationID,ChainID,Country,Segment
0,44,13,CZE,Value for money
1,45,6,CZE,Premium
2,46,23,CZE,Other
3,47,33,CZE,Premium
4,48,4,CZE,Premium


In [131]:
df_products = pd.read_csv('../assets/jobsheet-5/products.csv')
df_products.head()

,ProductID,Description
0,1,Rucní zadání
1,2,Nafta
2,3,Special
3,4,Super
4,5,Natural


In [132]:
df_transactions = pd.read_csv('../assets/jobsheet-5/transactions_1k.csv')
df_transactions.head()


,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05


In [133]:
df_yearmonth = pd.read_csv('../assets/jobsheet-5/yearmonth.csv')
df_yearmonth.head()

,CustomerID,Date,Consumption
0,5,201207,528.30
1,5,201302,1598.28
2,5,201303,1931.36
3,5,201304,1497.14
4,6,201203,51.06


## Data Understanding
- Merge tables
---





In [134]:
df_transactions['Date'] = pd.to_datetime(df_transactions['Date'])
df_yearmonth['Date'] = pd.to_datetime(df_yearmonth['Date'], format='%Y%m')
df_transactions['YearMonth'] = df_transactions['Date'].dt.to_period('M').dt.to_timestamp()

merged_df = df_transactions.merge(df_customer, on='CustomerID', how='left')
merged_df = merged_df.merge(df_gasstation, on='GasStationID', how='left', suffixes=('', '_GasStation'))
merged_df = merged_df.merge(df_products, on='ProductID', how='left')
merged_df = merged_df.merge(df_yearmonth, left_on=['CustomerID', 'YearMonth'], right_on=['CustomerID', 'Date'], how='left', suffixes=('', '_YearMonth'))

merged_df.drop(columns=['Date_YearMonth'], inplace=True)

merged_df.head()


,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price,YearMonth,Segment,Currency,ChainID,Country,Segment_GasStation,Description,Consumption
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64,2012-08-01,KAM,CZK,4,CZE,Premium,Nafta,142991.10
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72,2012-08-01,KAM,CZK,4,CZE,Premium,Nafta,123653.64
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99,2012-08-01,KAM,CZK,4,CZE,Premium,Provoz.nápl.,123653.64
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74,2012-08-01,KAM,CZK,30,CZE,Noname,Natural,18107.97
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05,2012-08-01,KAM,CZK,2,CZE,Noname,Nafta,72690.54


## Data Understanding
- Preview Data types
---





In [135]:
merged_df.dtypes

TransactionID                  int64
Date                  datetime64[ns]
Time                          object
CustomerID                     int64
CardID                         int64
GasStationID                   int64
ProductID                      int64
Amount                         int64
Price                        float64
YearMonth             datetime64[ns]
Segment                       object
Currency                      object
ChainID                        int64
Country                       object
Segment_GasStation            object
Description                   object
Consumption                  float64
dtype: object

## Soal-1
Buatlah table list Top 5 Customers yang memiliki nilai transaksi paling banyak 
---





In [136]:
top_customers = (merged_df.groupby('CustomerID')['Price'].sum().sort_values(ascending=False).head(5).reset_index())

top_customers = top_customers.merge(df_customer, on='CustomerID', how='left')
top_customers.columns = ['CustomerID', 'Total Transaction', 'Segment', 'Currency']

top_customers

,CustomerID,Total Transaction,Segment,Currency
0,19182,8618.54,KAM,CZK
1,16358,8400.94,KAM,CZK
2,31543,6161.50,KAM,CZK
3,12459,5974.47,KAM,CZK
4,30895,5940.01,KAM,CZK


## Soal-2
Buatlah list Top 5 Gas stations yang memiliki nilai transaksi paling banyak
---

In [137]:
top_gasstations = (merged_df.groupby('GasStationID')['Price'].sum().sort_values(ascending=False).head(5).reset_index())

top_gasstations = top_gasstations.merge(df_gasstation, on='GasStationID', how='left')
top_gasstations.columns = ['GasStationID', 'Total Transaction', 'ChainID', 'Country', 'Segment']

top_gasstations


,GasStationID,Total Transaction,ChainID,Country,Segment
0,4347,8557.94,4,CZE,Premium
1,48,6868.67,4,CZE,Premium
2,3250,6744.47,24,CZE,Discount
3,1155,6701.69,4,CZE,Premium
4,2417,6065.16,2,CZE,Noname


## Soal-3
Buatlah list Top 5 Jenis produk yang memiliki nilai transaksi paling banyak
---

In [138]:
top_products = (merged_df.groupby('ProductID')['Price'].sum().sort_values(ascending=False).head(5).reset_index())

top_products = top_products.merge(df_products, on='ProductID', how='left')
top_products.columns = ['ProductID', 'Total Transaction', 'Description']

top_products


,ProductID,Total Transaction,Description
0,2,345669.45,Nafta
1,9,27261.03,Diesel +
2,5,24639.46,Natural
3,20,7233.72,Nafta Plus
4,8,3999.92,Natural +


## Soal-4
Buatlah deskripsi statistik untuk masing masing hari (23,24,25,26) pada dataset transaction_1k
---

In [139]:
tanggal_list = [23, 24, 25, 26]
transaction_selected = merged_df[merged_df['Date'].dt.day.isin(tanggal_list)]

daily_stats = transaction_selected.groupby(transaction_selected['Date'].dt.day)[['Amount', 'Price']].describe()
daily_stats

Amount                                                        Price  \
      count       mean        std  min    25%   50%    75%    max  count   
Date                                                                       
23     10.0  29.600000  24.900692  1.0  21.25  26.5  35.25   91.0   10.0   
24    492.0  19.936992  21.949630  0.0   5.00  19.0  27.00  264.0  492.0   
25    425.0  18.880000  20.376114  0.0   4.00  17.0  26.00  154.0  425.0   
26     73.0  21.219178  18.932810  0.0   6.00  20.0  29.00  101.0   73.0   

                                                                        
            mean         std    min      25%     50%      75%      max  
Date                                                                    
23    685.379000  534.046487  76.42  481.170  619.44  795.095  1994.71  
24    409.939736  488.269481   1.76   76.125  332.45  615.395  5762.49  
25    426.940071  453.134207   6.09  102.220  370.38  600.850  3600.84  
26    487.431370  429.192499   4.74  142.840  437.03  675.520  2303.75

## Soal-5
Carilah waktu terbaik (hari dan jam) dimana paling banyak user gas station melakukan transaksi
---

In [140]:
merged_df['Jam'] = pd.to_datetime(merged_df['Time'], format='%H:%M:%S').dt.hour
merged_df['Hari'] = merged_df['Date'].dt.day

transaction_by_day_hour = (merged_df.groupby(['Hari', 'Jam'])['TransactionID'].count().reset_index().rename(columns={'TransactionID': 'Total Transaksi'}))

best_time = transaction_by_day_hour.sort_values(by='Total Transaksi', ascending=False).head(1)
best_time

,Hari,Jam,Total Transaksi
17,24,14,58


In [141]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Jam'] = pd.to_datetime(merged_df['Time'], format='%H:%M:%S').dt.hour
merged_df['Hari'] = merged_df['Date'].dt.strftime('%Y-%m-%d')

transaction_by_day_hour = (merged_df.groupby(['Hari', 'Jam']).size().reset_index(name='Total Transaksi'))
transaction_by_day_hour['Jam'] = transaction_by_day_hour['Jam'].astype(str).str.zfill(2) + ':00'

best_time = transaction_by_day_hour.sort_values(by='Total Transaksi', ascending=False).head(1)
best_time

,Hari,Jam,Total Transaksi
17,2012-08-24,14:00,58


## Soal-6
Dari segi bisnis understanding, tujuan utama dari analisis data customer ini adalah?
---

<ol>
    <li>Tujuan pertama adalah untuk mengetahui jenis produk dari gas station yang paling digemari oleh customer. Sehingga produsen dapat menambah stock terhadap jenis tersebut produk</li>
    <li>Tujuan kedua mengetahui stasiun gas mana yang memiliki penjualan paling banyak. Sehingga produsen dapat memberi stock lebih ke stasiun gas tersebut</li>
    <li>Tujuan terakhir untuk mengetahui jam berapa saja penjualan produk dibanjiri banyak customer, hal ini sangat membantu untuk mempersiapkan penjualan di jam-jam tersebut </li>
</ol> 